## Rozwiązywanie układów równań liniowych metodami interacyjnymi 

### Zadanie 1
Zaimplementuj metodę Jacobiego. Podaj warunki jej stosowalności. Wygeneruj co najmniej trzy odpowiednie macierze o różnych wielkościach i sprawdź działanie swojej metody. Zwróć uwagę na zbieżność tej metody. 



In [0]:
import numpy as np
import random


def jacobi_solver(A, b, N, eps):
    iter_count = 0
    x = np.zeros_like(b)
    for _ in range(N):
        iter_count += 1
        x_new = np.zeros_like(x)

        for i in range(A.shape[0]):
            x_new[i] = (b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

        if np.allclose(x, x_new, atol=eps, rtol=0.):
            return [x_new, iter_count]
        x = x_new
    return [x, iter_count]


Warunek konieczny i dostateczny zbieżności jest spełniony m.in. gdy
**A** jest **nieredukowalna** i **diagonalnie dominująca**.  
Macierz **A** jest **nieredukowalna**, jeżeli poprzez przestawienie wierszy
i kolumn nie można jej sprowadzić do postaci blokowej górnej trójkątnej.    

Macierz **A** o wymiarze n x n nazywamy **diagonalnie dominującą**, jeśli
dla i=1,2,...,n zachodzi nierówność          
$|a_{ii}| \geq \sum_{j=1, j{\neq}i}^n |a_{ij}|$

W celu przetestowania metody Jacobiego stworzyłem dodatkową funkcja, która generuje macierz spełniająca powyższe warunki : 

In [0]:
def create_rand_matrix(min, max, N):
    A = np.zeros((N, N))
    for i in range(N):
        sum = 0
        for j in range(N):
            if j >= i: A[i][j] = A[j][i] = random.randint(min, max)
            if i != j: sum += abs(A[i][j])
        A[i][i] = sum
    return A


def create_rand_vector(min, max, N):
    return np.random.uniform(low=min, high=max + 1, size=(N,))

In [125]:
def compare_results(A, b, N, eps):
    print("Numpy solution:")
    print(np.linalg.solve(A, b))
    print("Jacobi solver solution :")
    print(jacobi_solver(A, b, N, eps)[0])


A = create_rand_matrix(-15, 15, 5)
b = create_rand_vector(-10, 10, 5)
compare_results(A, b, 1000, 1e-10)

A = create_rand_matrix(-30, 30, 10)
b = create_rand_vector(-20, 20, 10)
compare_results(A, b, 1000, 1e-10)

A = create_rand_matrix(-50, 50, 15)
b = create_rand_vector(-20, 20, 15)
compare_results(A, b, 1000, 1e-10)

Numpy solution:
[-0.17473389 -0.38524748 -0.045583    0.49376269  0.21841752]
Jacobi solver solution :
[-0.17473389 -0.38524748 -0.045583    0.49376269  0.21841752]
Numpy solution:
[-0.00604408  0.17384578  0.12206283 -0.0012129  -0.06486635 -0.11330034
  0.08838833 -0.01760385  0.03724974  0.14555495]
Jacobi solver solution :
[-0.00604408  0.17384578  0.12206283 -0.0012129  -0.06486635 -0.11330034
  0.08838833 -0.01760385  0.03724974  0.14555495]
Numpy solution:
[-0.02828609 -0.01310856  0.00716221 -0.083047    0.05282158  0.04126928
 -0.05637648 -0.03280094 -0.04460144 -0.01078921  0.00076575 -0.03387245
 -0.05972013  0.08359163 -0.04903926]
Jacobi solver solution :
[-0.02828609 -0.01310856  0.00716221 -0.083047    0.05282158  0.04126928
 -0.05637648 -0.03280095 -0.04460144 -0.01078921  0.00076575 -0.03387245
 -0.05972013  0.08359163 -0.04903926]


Z powyższych wyników, możemy wywnioskować, że zaimplementowana przez nas metoda Jacobiego działa poprawnie (wyniki z dedykowanej biblioteki numpy są identyczne z wynikami z implementacji metody Jacobiego).

### Zadanie 2
Zaimplementuj metodę Gaussa-Seidla i kolejnych nadrelaksacji (successive over-relaxation). Podaj warunki stosowalności tych metod. Przeprowadź badanie działania swoich implementacji analogicznie jak w poprzednim zadaniu. Porównaj zbieżność wszystkich trzech metod. 

In [0]:
def gauss_siedla_solver(A, b, N, eps):
    iter_count = 0
    x = np.zeros_like(b)
    for _ in range(N):
        iter_count += 1
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            x_new[i] = (b[i] - np.dot(A[i, :i], x_new[:i]) - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

        if np.allclose(x, x_new, atol=eps, rtol=0.):
            return [x_new, iter_count]
        x = x_new
    return [x, iter_count]


def sor_solver(A, b, omega, N, eps):
    iter_count = 0
    x = np.zeros_like(b)
    if omega < 0 or omega > 2:
        print('omega should be inside (0, 2)')
        return [x, -1]
    n = b.shape
    x_new = np.zeros_like(x)
    for _ in range(N):
        iter_count += 1
        for i in range(n[0]):
            new_values_sum = np.dot(A[i, :i], x[:i])
            old_values_sum = np.dot(A[i, i + 1:], x_new[i + 1:])
            x[i] = (b[i] - (old_values_sum + new_values_sum)) / A[i, i]
            x[i] = np.dot(x[i], omega) + np.dot(x_new[i], (1 - omega))
        if np.linalg.norm(np.dot(A, x) - b) < eps:
            return [x_new, iter_count]
        x_new = x
    return [x, iter_count]


Dla **metody Jacobiego** warunek konieczny, i dostateczny zbieżności jest spełniony m.in. gdy
macierz **A** jest **nieredukowalna** i **diagonalnie dominująca**.  
Macierz **A** jest **nieredukowalna**, jeżeli poprzez przestawienie wierszy
i kolumn nie można jej sprowadzić do postaci blokowej górnej trójkątnej.    

Macierz **A** o wymiarze n x n nazywamy **diagonalnie dominującą**, jeśli
dla i=1,2,...,n zachodzi nierówność          
$|a_{ii}| \geq \sum_{j=1, j{\neq}i}^n |a_{ij}|$       

        
**Metoda Gaussa-Seidela**, jako ulepszenie metody Jacobiego, zachowuje
te same warunki zbieżności. Jeżeli macierz A jest dodatnio określona to
metoda Gaussa-Seidela jest zbieżna dla dowolnego wektora
początkowego [8, 10].          
Metodę Gaussa-Seidela stosuje się niemal wyłącznie do układów
z macierzą diagonalnie **dominującą** gdyż w wielu praktycznych
zastosowaniach jest to łatwy do spełnienia warunek gwarantujący
zbieżność metody.   
   
**Metoda kolejnych nadrelaksacji (SOR)** jest to modyfikacja metody Gaussa-Seidela(warunki te same) przyspieszająca zbieżność
konstruowanego ciągu, polega na przemnożeniu poprawki
obliczanej przez odpowiednio dobraną liczbę $\omega$.     
Zwiększając współczynnik $\omega$ , można próbować przyspieszać jej
zbieżność. Parametr $\omega$ może przyjmować wartości co najwyżej
z przedziału (0, 2), gdyż dla pozostałych wartości metoda może nie być
zbieżna dla pewnych przybliżeń początkowych.     
  
Metody Jacobiego i Gaussa-Seidela można ewentualnie stosować do
układów bardzo dobrze uwarunkowanych. Znacznie efektywniejsze,
szczególnie dla zadań o dużym wskaźniku uwarunkowania, jest użycie
metody SOR.   

Test poprawności zaimplementowanych rozwiązań : 

In [127]:
def compare_results(A, b, N, eps, omega):
    print("Numpy solution:")
    print(np.linalg.solve(A, b))
    print("Gauss-Siedla solver solution :")
    print(gauss_siedla_solver(A, b, N, eps)[0])
    print("SOR solver solution :")
    print(sor_solver(A, b, omega, N, eps)[0])


A = create_rand_matrix(-15, 15, 5)
b = create_rand_vector(-10, 10, 5)
compare_results(A, b, 1000, 1e-10, 0.2)

A = create_rand_matrix(-30, 30, 10)
b = create_rand_vector(-20, 20, 10)
compare_results(A, b, 1000, 1e-10, 0.2)

A = create_rand_matrix(-50, 50, 25)
b = create_rand_vector(-20, 20, 25)
compare_results(A, b, 1000, 1e-10, 0.2)

Numpy solution:
[ 0.05738142  0.16928196 -0.32255261 -0.12568326 -0.51333213]
Gauss-Siedla solver solution :
[ 0.05738142  0.16928196 -0.32255261 -0.12568326 -0.51333213]
SOR solver solution :
[ 0.05738142  0.16928196 -0.32255261 -0.12568326 -0.51333213]
Numpy solution:
[-0.11943933 -0.10222989  0.01897715  0.1461937   0.0292072   0.1010488
 -0.02913119 -0.18115142 -0.12465763  0.06894863]
Gauss-Siedla solver solution :
[-0.11943933 -0.10222989  0.01897715  0.1461937   0.0292072   0.1010488
 -0.02913119 -0.18115142 -0.12465763  0.06894863]
SOR solver solution :
[-0.11943933 -0.10222989  0.01897715  0.1461937   0.0292072   0.1010488
 -0.02913119 -0.18115142 -0.12465763  0.06894863]
Numpy solution:
[ 0.02179042 -0.03687972 -0.02531252  0.0287093  -0.01179903 -0.02493491
  0.01072436 -0.02668894  0.02116012  0.03959596  0.02395876 -0.0053013
 -0.03725647  0.01479283 -0.02056916 -0.0191237  -0.03817169 -0.00070036
 -0.01938442 -0.01272447 -0.01481525  0.00435948  0.0179912   0.01930233
 -0

Z powyższych wyników, możemy wywnioskować, że zaimplementowane przez nas metody działają poprawnie (wyniki z dedykowanej biblioteki numpy są identyczne z wynikami z zaimplementowanych metod).

Porówanie zbieżności powyższych metod :

In [0]:
def compare_iter_counters(A, b, N, eps, omega):
    print("Jacobi solver iterration counter :")
    print(jacobi_solver(A, b, N, eps)[1])
    print("Gauss-Siedla solver iterration counter :")
    print(gauss_siedla_solver(A, b, N, eps)[1])
    print("SOR solver iterration counter :")
    print(sor_solver(A, b, omega, N, eps)[1])

Testy przeprowadzilem dla $\omega$ = 1.9 oraz losowanych wartości [-15,150]

Porównanie ilości iteracji dla macierzy 25x25: 

In [129]:
A = create_rand_matrix(-15, 150, 25)
b = create_rand_vector(-15, 150, 25)
compare_iter_counters(A, b, 1000, 1e-10, 1.9)

Jacobi solver iterration counter :
847
Gauss-Siedla solver iterration counter :
15
SOR solver iterration counter :
19


Porównanie ilości iteracji dla macierzy 50x50 : 

In [130]:
A = create_rand_matrix(-15, 150, 50)
b = create_rand_vector(-15, 150, 50)
compare_iter_counters(A, b, 1000, 1e-10, 1.9)

Jacobi solver iterration counter :
966
Gauss-Siedla solver iterration counter :
14
SOR solver iterration counter :
20


Porównanie ilości iteracji dla macierzy 100x100: 

In [131]:
A = create_rand_matrix(-15, 150, 100)
b = create_rand_vector(-15, 150, 100)
compare_iter_counters(A, b, 1000, 1e-10, 1.9)

Jacobi solver iterration counter :
874
Gauss-Siedla solver iterration counter :
14
SOR solver iterration counter :
20
